# Lab 5 Part 1 - Telephone game

First, set up a Gemini API key from aistudio.google.com. You can freely access Gemini model APIs as long as your usage is low, and as long as you don't mind Google collecting your data for training. I believe that is acceptable for the purposes of this lab. In general however, please be wary about passing your own data to LLM providers.

Next, paste the API key in a file called `googleapikey.txt` in the same folder as this notebook. It should look like this:

```
# googleapikey.txt
your-api-key-here
```

Make sure you have these packages installed:`langchain-google-genai`, `deepeval`

## Setup

In [20]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [21]:
with open("googleapikey.txt", "r") as f:
    GOOGLE_API_KEY = f.read()

## One-turn text translation

Here is sample code to set up text translation using a Gemini model using tools from langchain:

In [22]:
# Initialize a single model
llm = ChatGoogleGenerativeAI(temperature=1.5, model="gemini-2.0-flash-lite", api_key=GOOGLE_API_KEY)

# Create a simple translation prompt
prompt = ChatPromptTemplate.from_template(
    "Translate the following text to {target_language}. "
    "Only return the translation, nothing else:\n\n{text}"
)

# Create a simple chain
chain = prompt | llm | StrOutputParser()

In [23]:
# Test with one translation
original_text = "The quick brown fox jumps over the lazy dog."

result = chain.invoke({
    "text": original_text,
    "target_language": "Korean"
})

In [24]:
original_text

'The quick brown fox jumps over the lazy dog.'

In [25]:
result

'빠른 갈색 여우가 게으른 개를 뛰어넘는다.'

In [26]:
chain.invoke({
    "text": result,
    "target_language": "English"
})

'The quick brown fox jumps over the lazy dog.'

## Telephone game

The telephone game as per wikipedia:

> Players form a line or circle, and the first player comes up with a message and whispers it to the ear of the second person in the line. The second player repeats the message to the third player, and so on. When the last player is reached, they announce the message they just heard, to the entire group. The first person then compares the original message with the final version. Although the objective is to pass around the message without it becoming garbled along the way, part of the enjoyment is that, regardless, this usually ends up happening. Errors typically accumulate in the retellings, so the statement announced by the last player differs significantly from that of the first player, usually with amusing or humorous effect. Reasons for changes include anxiousness or impatience, erroneous corrections, or the difficult-to-understand mechanism of whispering.

Get LLMs to perform this translation chain, cycling through the official languages of the United Nations, but starting with and ending with English: Arabic -> French -> Russian -> Spanish

Use this text chunk:

```
The Monarch butterfly is one of the most recognizable and iconic insects in North America, with its distinctive orange and black wings featuring striking white spots. Each year, these magnificent creatures migrate thousands of miles from Canada and the United States to their wintering grounds in Mexico, a journey that is both breathtaking and awe-inspiring.
```

When you are done, see how different the translation was: (1) by visual inspection, and (2) by using an automated evaluator using LLMs as below:

In [27]:
from deepeval import evaluate
from deepeval.metrics import AnswerRelevancyMetric
from deepeval.test_case import LLMTestCase
from deepeval.models import GeminiModel

In [28]:
def evaluate_telephone_game(original_text: str, final_text: str):
    model = GeminiModel(
        model_name="gemini-2.0-flash-lite",
        api_key=GOOGLE_API_KEY,
        temperature=0
    )
    # Create test case
    test_case = LLMTestCase(
        input=f"Preserve the meaning of: {original_text}",
        actual_output=final_text,
        expected_output=original_text
    )

    relevancy = AnswerRelevancyMetric(threshold=0.5, model=model, async_mode=False)
    relevancy.measure(test_case)
    
    return relevancy.score

# 1-step

In [29]:
monarchbutterfly = "The Monarch butterfly is one of the most recognizable and iconic insects in North America, with its distinctive orange and black wings featuring striking white spots. Each year, these magnificent creatures migrate thousands of miles from Canada and the United States to their wintering grounds in Mexico, a journey that is both breathtaking and awe-inspiring."

s1 = chain.invoke({
    "text": monarchbutterfly,
    "target_language": "Arabic"
})

s1

'فراشة الملك هي واحدة من الحشرات الأكثر شهرة وأيقونية في أمريكا الشمالية، بأجنحتها البرتقالية والسوداء المميزة التي تتميز ببقع بيضاء ملفتة. كل عام، تهاجر هذه المخلوقات الرائعة آلاف الأميال من كندا والولايات المتحدة إلى مناطق بياتها الشتوي في المكسيك، وهي رحلة مذهلة ومثيرة للإعجاب.'

In [30]:
evaluate_telephone_game(monarchbutterfly, s1)

Output()

1.0

# 2-step

In [31]:
s2 = chain.invoke({
    "text": s1,
    "target_language": "French"
})

s2

"Le monarque est l'un des insectes les plus célèbres et emblématiques d'Amérique du Nord, avec ses ailes orange et noires distinctives ornées de taches blanches frappantes. Chaque année, ces créatures magnifiques migrent sur des milliers de kilomètres du Canada et des États-Unis vers leurs sites d'hivernage au Mexique, un voyage extraordinaire et impressionnant."

In [32]:
evaluate_telephone_game(monarchbutterfly, s2)

Output()

1.0

# 3-step

In [33]:
s3 = chain.invoke({
    "text": s2,
    "target_language": "Russian"
})

s3

'Монарх — одно из самых известных и знаковых насекомых Северной Америки, с его отличительными оранжевыми и черными крыльями, украшенными поразительными белыми пятнами. Каждый год эти великолепные существа мигрируют на тысячи километров из Канады и Соединенных Штатов в свои места зимовки в Мексике — экстраординарное и впечатляющее путешествие.'

In [34]:
evaluate_telephone_game(monarchbutterfly, s3)

Output()

1.0

# 4-step

In [35]:
s4 = chain.invoke({
    "text": s3,
    "target_language": "Spanish"
})

s4

'La monarca es uno de los insectos más conocidos e icónicos de América del Norte, con sus distintivas alas naranjas y negras adornadas con llamativas manchas blancas. Cada año, estas magníficas criaturas migran miles de kilómetros desde Canadá y Estados Unidos hasta sus lugares de invernación en México, un viaje extraordinario e impresionante.'

In [36]:
evaluate_telephone_game(monarchbutterfly, s4)

Output()

1.0

# back to English

In [37]:
s5 = chain.invoke({
    "text": s4,
    "target_language": "English"
})

s5

'The monarch is one of the most well-known and iconic insects of North America, with its distinctive orange and black wings adorned with striking white spots. Each year, these magnificent creatures migrate thousands of kilometers from Canada and the United States to their overwintering sites in Mexico, an extraordinary and impressive journey.'

In [38]:
evaluate_telephone_game(monarchbutterfly, s5)

Output()

1.0

## EX: How good is the translation quality at each step?

To be more clear, first step translation means English -> Arabic -> English. 2nd step translation means English -> Arabic -> French -> English.

Perfect.